In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train_metadata = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
df_test_metadata = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
df_train_metadata.head

In [ ]:
import tensorflow as tf
import cv2
import missingno as msno
import matplotlib.pyplot as plt

train_image = df_train_metadata.copy()
test_image = df_test_metadata.copy()

train_image["file_path"] = df_train_metadata["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/train/" + x + ".jpg")
test_image["file_path"] = df_test_metadata["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/test/" + x + ".jpg")



plt.figure(figsize=(20, 20))
row, col = 5, 4
for i in range(row * col):
    plt.subplot(row, col, i+1)
    image = cv2.imread(train_image.loc[i, 'file_path'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    target = train_image.loc[i, 'Pawpularity']
    plt.imshow(image)
    plt.title(f"No: {i}" f"   Pawpularity: {target}")
plt.show()

In [ ]:
#load data google drive
image_features_train = pd.read_csv('../input/image-features/image_features_train.csv')
image_features_test = pd.read_csv('../input/image-features/image_features_test.csv')


df_train = pd.concat([df_train_metadata['Pawpularity'], df_train_metadata.loc[:,'Subject Focus':'Blur'],image_features_train.loc[:,'centroid_pr_x':'kurtosis_pr_y']], axis=1)
df_test = pd.concat([df_test_metadata.loc[:,'Subject Focus':'Blur'],image_features_test.loc[:,'centroid_pr_x':'kurtosis_pr_y']], axis=1)

df_test.head(3)

In [ ]:
#!pip install skfeature-chappers

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
#ambil kolom untuk label diskrit (class)
label_diskrit = np.asarray(df_train['Pawpularity'])

# #ambil kolom untuk label kontinu
label_kontinu = np.asarray(df_train['Pawpularity'])

# # ambil kolom untuk features
features_train = np.asarray(df_train.loc[:, 'Subject Focus':'kurtosis_pr_y'])
features_test = np.asarray(df_test.loc[:, 'Subject Focus':'kurtosis_pr_y'])
# #melakukan label encoding label diskrit
# # encoder = preprocessing.LabelEncoder().fit(label_diskrit)
# # transformed_label_diskrit = encoder.transform(label_diskrit)
# # print(transformed_label_diskrit)

# melakukan feature scaling
scaler = preprocessing.MinMaxScaler(feature_range=(0, 10)).fit(features_train)
scaled_feature_train = scaler.transform(features_train)
scaled_features_test = scaler.transform(features_test)
#print(scaled_feature_train)


# # bentuk awal method dan parameter gini-index terdapat X dan y
# # dimana X adalah fitur dan y adalah label
# # MIM.mim(X, y)
# # ranked_index = fisher_score.fisher_score(scaled_feature, transformed_label_diskrit, mode='raw')
# # print("\nfisher score raw")
# # print(ranked_index)

# # ranked_index = fisher_score.fisher_score(scaled_feature, transformed_label_diskrit, mode='rank')
# # print("\nfisher score rank")
# # print(ranked_index)

# ranked_index = JMI.jmi(scaled_feature_train, label_kontinu, mode='index')
# print("\nJMI")
# print(ranked_index)


# # menampilkan fitur yang sudah diurutkan berdasarkan seleksi fitur
# result_train = scaled_feature_train[:, ranked_index[:]]
# print(result_train)
# result_test = scaled_features_test[:, ranked_index[:]]
#isi jawaban di sini menggunakan CIFE
# print(scaled_feature)


In [ ]:
ranked_index = [12,8,10,1,3,6,9,7,5,2,11,4,0,13]
print("\nCIFE")
print(ranked_index)

# menampilkan fitur yang sudah diurutkan berdasarkan seleksi fitur
result_train = scaled_feature_train[:, ranked_index[:]]
print(result_train)
result_test = scaled_features_test[:, ranked_index[:]]

In [ ]:
def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)

    distance = predict - actual

    square_distance = distance ** 2

    mean_square_distance = square_distance.mean()

    score = np.sqrt(mean_square_distance)

    return score

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

jumlah_fitur = range(1,scaled_feature_train.shape[1]+1)
scores = []
score = 1000
best_score = 1000
best_feature_number = 0

#Mencoba optimasi hyerparameter untuk setiap kombinasi/jumlah fitur
for jumlah_fitur_terbaik in jumlah_fitur:
    #print(jumlah_fitur_terbaik)
    selected_features = result_train[:,0:jumlah_fitur_terbaik]
    #split data training dan data testing
    X_train, X_test, y_train, y_test = train_test_split(selected_features, label_kontinu, test_size=0.3, random_state=0)

    regressor = DecisionTreeRegressor()
    #regressor = KNeighborsRegressor()
    #regressor = SVR()
    #regressor = GradientBoostingRegressor()
    #regressor = xgb.XGBRegressor()
    #regressor = AdaBoostRegressor()
    
    # optimasi hyperparameter
    param_grid = [
    #{'n_neighbors':[3,5,7,9,11,13,15], 'metric':['euclidean','manhattan','chebyshev','minkowski','wminkowski','seuclidean','mahalanobis']}
    #{'C': [1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001], 'kernel': ['rbf']},
    #{'n_estimators': [50, 100, 150, 200], 'learning_rate': [0.5, 1, 1.5, 2], 'subsample': [0.6, 0.8, 1.0], 'max_depth': [5, 10, 15, 20]}
    {'max_depth':[5, 10, 15], 'min_samples_split':[0.1, 1.0, 10], 'min_samples_leaf':[0.1, 0.5, 5]}
    #{'n_estimators': [50, 100, 150, 200],'learning_rate': [0.1,0.2,0.3],}
    #{'max_depth': [3, 5, 6, 10, 15, 20], 'learning_rate': [0.01, 0.1, 0.2, 0.3], 'subsample': np.arange(0.5, 1.0, 0.1), 'colsample_bytree': np.arange(0.4, 1.0, 0.1), 'colsample_bylevel': np.arange(0.4, 1.0, 0.1), 'n_estimators': [100, 500, 1000]}
    ]
    
    #menentukan prioritas scoring menggunakan apa (accuracy/precision/recall, dll)
    metric = make_scorer(rmse, greater_is_better = False)

    model = RandomizedSearchCV(regressor, param_grid, scoring=metric, cv=5, refit = True, verbose = 3) 
    # fitting the model for grid search 
    model.fit(X_train, y_train)

    # print best parameter after tuning 
    print(model.best_params_) 
      
    # print how our model looks after hyper-parameter tuning 
    print(model.best_estimator_)

    #model_predictions = model.predict(X_test) 

    #model.fit(X_train, y_train)
    score = abs(model.score(X_test, y_test))
    scores.append(score)

    #menentukan model terbaik berdasarkan score terbaik menggunakan kombinasi jumlah fitur dan optimasi hyperparameter
    if(best_score > score):
      best_score = score
      best_model = model
      best_feature_number = jumlah_fitur_terbaik
      best_parameter = model.best_params_
      
      #menyimpan best_X_test dengan jumlah fitur terbaik
      best_X_test = X_test


plt.figure()
plt.xlabel('jumlah_fitur_terbaik')
plt.ylabel('score')
plt.scatter(jumlah_fitur, scores)
plt.grid()

print(scores);
print('Jumlah fitur terbaik adalah: ',best_feature_number)
print('Score terbaik adalah: ',best_score)
print('Parameter terbaik adalah: ',best_parameter)

final_predictions = best_model.predict(best_X_test) 
  
# print classification report 
# from sklearn.metrics import classification_report
# print(classification_report(y_test, final_predictions)) 

# plot confusion matrix
# from sklearn.metrics import plot_confusion_matrix
# plot_confusion_matrix(best_model, best_X_test, y_test)  
# plt.show()

In [ ]:
#isi jawaban plot regresi dt
from sklearn.metrics import r2_score

fig, ax = plt.subplots()
ax.text(1, 9.5,'$R^2=$'+str(round(r2_score(y_test, final_predictions),4)), fontsize=12, verticalalignment='top', multialignment='center')
ax.text(1, 11,'$MSE=$'+str(round(rmse(y_test, final_predictions),4)), fontsize=12, verticalalignment='top', multialignment='center')

ax.set_xlim(xmin=1)
ax.set_ylim(ymin=1)
ax.set_xlim(xmax=100)
ax.set_ylim(ymax=100)

ax.set_xlabel('Actual Value', fontsize=14)
ax.set_ylabel('Predicted Value', fontsize=14)
ax.scatter(y_test, final_predictions, s=100, c=y_test, cmap='viridis')

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

# now plot both limits against eachother
ax.plot(lims, lims, 'r--', alpha=0.75, zorder=0)
ax.set_aspect('equal')
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.grid(True, which='both')

xvalue = np.linspace(1,10,10)
lsigma = ax.fill_between(xvalue, xvalue+1, xvalue-1, color='blue', alpha=0.3)

plt.show()

In [ ]:
#generate file prediksi
selected_features_kaggle_test = result_test[:, 0:best_feature_number]
final_predictions_kaggle = best_model.predict(selected_features_kaggle_test) 
final_predictions_kaggle

df_hasil = pd.DataFrame({"Id":df_test_metadata.Id,"Pawpularity":final_predictions_kaggle})

df_hasil.to_csv('submission.csv', index=False)
df_hasil.head()